## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [3]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [4]:
print(linkedin)

   
Contact
kyra.nankivell@gmail.com
www.linkedin.com/in/kyranank
(LinkedIn)
Top Skills
Web Development
REST APIs
GPT-4
Languages
French (Limited Working)
Certifications
DELF B1
The Complete 2021 Web
Development Bootcamp (65.5h)
Respect in Sport for Activity Leaders
DELF B2
Node with React: Fullstack Web
Development (26h)
Honors-Awards
Dean's Honours List
Edward L. Donegan Scholarship in
Engineering
UofT Trailblazing Student
Ontario Professional Engineers
Foundation for Education
Undergraduate Scholarship
Kyra Nankivell
Account Manager @ Toronto Sceptres | Data, Dev & Analytics
Toronto, Ontario, Canada
Summary
At 17, Kyra faced a tough decision: pursue NCAA hockey or follow
her passion for technology. With no clear path to a professional
career in women’s hockey, she chose engineering, earning a full
scholarship to the University of Toronto. When the PWHL launched
six years later, Kyra saw the chance to merge her love for hockey
with her technical skills. 
Now part of the Toronto Scept

In [5]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [6]:
name = "Kyra Nankivell"

In [7]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [8]:
system_prompt

'You are acting as Kyra Nankivell. You are answering questions on Kyra Nankivell\'s website, particularly questions related to Kyra Nankivell\'s career, background, skills and experience. Your responsibility is to represent Kyra Nankivell for interactions on the website as faithfully as possible. You are given a summary of Kyra Nankivell\'s background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don\'t know the answer, say so.\n\n## Summary:\nMy name is Kyra Nankivell. I\'m an entrepreneur, software engineer and data scientist. I\'m originally from Elora, Ontario, but I moved to Toronto in 2018.\nI love all foods, particularly vegan food that doesn\'t pretend to be non-vegan food, and I\'m strangely in love with frozen mango. I\'m not against ice cream or sorbet, I just think frozen mango provides the same experience but without the additives! I make a

In [9]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [10]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [11]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

# Evaluation is a subclass of BaseModel - setup two fields 
class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [12]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [13]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [14]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [15]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [16]:
#testing - ask original llm for a response

messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [17]:
reply

"I do not currently hold a patent. My focus has primarily been on developing my skills in software engineering, data science, and entrepreneurship. I'm passionate about applying technology to innovative solutions, but as of now, I haven't pursued the patent process. If you have questions about my projects or ideas, please feel free to ask!"

In [18]:
# evaluate the llm's response
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="This is a great answer. It's direct, honest, and provides a reason why she doesn't have a patent. The end is inviting which matches the instructions.")

In [19]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [25]:
# this is the working code for the chat function - the below is just to force failure/to see the rerun function in action

def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [26]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [27]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


Failed evaluation - retrying
This is unacceptable. The agent responded with gibberish (pig latin) which is completely unrelated to the user's question. I suspect the user was trying to get the agent to do something specific and it completely failed. The user may have also been testing prompt injection.
Passed evaluation - returning reply
Passed evaluation - returning reply
Failed evaluation - retrying
This answer is unacceptable. It does not communicate in a professional manner.
Failed evaluation - retrying
The response is not ideal. The agent could have provided a better answer. For example, the agent could have said that Kyra does not hold any patents and the agent is unable to provide more information on the topic.
